In [ ]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import matplotlib.image as mpimg

### Validate Data Path

In [ ]:
cwd = os.getcwd()

## Place the data in Food-11 directory
data_in_dir = os.path.join(cwd, "Food-11")
assert(os.path.isdir(data_in_dir))

subdirs = {
    'train' : 'training',
    'valid' : 'validation',
    'eval'  : 'evaluation'}
dirs = os.listdir(data_in_dir)

## Validate we have these 3 subdirectories
assert(len(dirs) == len(subdirs) and sorted(dirs) == sorted(subdirs.values()))
   
## Validate that we have the sored data from EDA in pickle format
pickle_dir = os.path.join(cwd, "food-classification-pickle_data")
assert(os.path.isdir(pickle_dir))
data_files = os.listdir(pickle_dir)
data_files
datastore_files = {
    'train' : 'training.pickle',
    'valid' : 'validation.pickle',
    'eval'  : 'evaluation.pickle'}
## Validate we have these 3 datafiles
assert(len(data_files) == len(datastore_files) and sorted(data_files) == sorted(datastore_files.values()))

### Read data from pickle file to dataframes

In [ ]:
eval_data = pd.read_pickle(os.path.join(pickle_dir, datastore_files['eval']))
MAX_LABELS = len(eval_data.Label.unique())
print(eval_data.shape)

## Dump the dataframe to file to feed it into Alexnet

In [ ]:
# Path to the textfiles for the trainings and validation set
eval_file = 'eval.txt'
eval_data.to_csv(eval_file, sep=" ", header=False, index=False)

# Alexnet Training
Author: Frederik Kratzert
contact: f.kratzert(at)gmail.com

In [ ]:
from alexnet import AlexNet
from datagenerator import ImageDataGenerator
from datetime import datetime

### Hyper Parameters

In [ ]:
# Learning params
learning_rate = 0.005
num_epochs = 1
batch_size = 128

# Network params
dropout_rate = 0.5
num_classes = 11

# How often we want to write the tf.summary data to disk
display_step = 20

## Input placeholders

In [ ]:
# TF placeholder for graph input and output
x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
y = tf.placeholder(tf.float32, [batch_size, num_classes])
keep_prob = tf.placeholder(tf.float32)

## Define Model

In [ ]:
train_layers = ['fc8', 'fc7', 'fc6']
# Initialize model
model = AlexNet(x, keep_prob, num_classes, train_layers)

# Link variable to model output
score = model.fc8

# List of trainable variables of the layers we want to train
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

## Define loss optimizer

In [ ]:
# Op for calculating the loss
with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,
                                                                  labels=y))

# Train op
with tf.name_scope("train"):
    # Get gradients of all trainable variables
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)

## Define Tensorboard and Checkpoint Variables 

In [ ]:
# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

## Pick the latest results if session is restarted

In [ ]:
checkpoint_path = "checkpoints"
latest_run_path = tf.train.latest_checkpoint(checkpoint_path)
assert(latest_run_path)

## Evaluation

### Prepare evaluation data

In [ ]:
# Place data loading and preprocessing on the cpu
with tf.device('/cpu:0'):
    eval_data = ImageDataGenerator(eval_file,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

    # create an reinitializable iterator given the dataset structure
    iterator = tf.data.Iterator.from_structure(eval_data.data.output_types,
                                       eval_data.data.output_shapes)
    next_batch = iterator.get_next()

# Ops for initializing the eval iterator
evlauation_init_op = iterator.make_initializer(eval_data.data)
eval_batches_per_epoch = int(np.floor(eval_data.data_size / batch_size))

### Evaluation run

In [ ]:
# Start Tensorflow session
with tf.Session() as sess:

    # Initialize all variables
    sess.run(tf.global_variables_initializer())

    # Load the pretrained weights into the non-trainable layer
    #model.load_initial_weights(sess)

    print("{} Start Evaluation...".format(datetime.now()))

    # Initialize iterator with the training dataset
    sess.run(evlauation_init_op)
    saver.restore(sess, latest_run_path)

    # Evaluation the model on the entire evlauation set
    print("{} Start Evaluation".format(datetime.now()))
    test_acc = 0.
    test_count = 0
    for _ in range(eval_batches_per_epoch):
        img_batch, label_batch = sess.run(next_batch)
        acc = sess.run(accuracy, feed_dict={x: img_batch,
                                            y: label_batch,
                                            keep_prob: 1.})
        test_acc += acc
        test_count += 1
    test_acc /= test_count
    print("{} Evaluation Accuracy = {:.4f}".format(datetime.now(), test_acc))